# Build the LSLGA parent sample

The purpose of this notebook is to build the parent sample for the Legacy Survey Large Galaxy Atlas (LSLGA).

### Imports and paths

In [13]:
import os
import numpy as np

import matplotlib.pyplot as plt

from legacypipe.survey import ccds_touching_wcs, LegacySurveyData

In [12]:
plt.style.use('seaborn-talk')
%matplotlib inline

In [14]:
dr = 'dr5'
LSLGAdir = os.getenv('LSLGA_DIR')

### Read the parent HyperLeda catalog

In [22]:
def _catalog_template(nobj=1):
    cols = [
        ('GALAXY', 'S28'), 
        ('PGC', 'S10'), 
        ('RA', 'f8'), 
        ('DEC', 'f8'),
        ('TYPE', 'S8'),
        ('MULTIPLE', 'S1'),
        ('RADIUS', 'f4'),
        ('BA', 'f4'),
        ('PA', 'f4'),
        ('BMAG', 'f4'),
        ('IMAG', 'f4'),
        ('VHELIO', 'f4'),
        ('BRICKNAME', 'S17', (4,))
        ]
    catalog = Table(np.zeros(nobj, dtype=cols))
    catalog['RADIUS'].unit = 'arcsec'
    catalog['VHELIO'].unit = 'km/s'

    return catalog

In [23]:
def read_leda(largedir='.', d25min=0.0, d25max=1000.0, decmin=-90.0, 
              decmax=+90.0, ramin=0.0, ramax=360.0):
    """Read the parent LEDA catalog and put it in a standard format.
    
    """
    cat = fits.getdata(os.path.join(largedir, 'sample', 'leda-logd25-0.05.fits.gz'), 1)

    outcat = _catalog_template(len(cat))
    outcat['GALAXY'] = cat['GALAXY']
    outcat['PGC'] = cat['PGC']
    outcat['RA'] = cat['RA']
    outcat['DEC'] = cat['DEC']
    outcat['TYPE'] = cat['TYPE']
    outcat['MULTIPLE'] = cat['MULTIPLE']
    outcat['RADIUS'] = cat['D25']/2.0 # semi-major axis radius [arcsec]
    outcat['BA'] = cat['BA']
    outcat['PA'] = cat['PA']
    outcat['BMAG'] = cat['BMAG']
    outcat['IMAG'] = cat['IMAG']
    outcat['VHELIO'] = cat['VHELIO']

    these = np.where((outcat['RADIUS']*2/60.0 <= d25max) *
                     (outcat['RADIUS']*2/60.0 >= d25min) *
                     (outcat['DEC'] <= decmax) *
                     (outcat['DEC'] >= decmin) *
                     (outcat['RA'] <= ramax) *
                     (outcat['RA'] >= ramin)
                     )[0]
    outcat = outcat[these] 

    return outcat

In [17]:
survey = LegacySurveyData()

In [21]:
bricks = survey.get_bricks()
allccds = survey.get_annotated_ccds()
cut = survey.photometric_ccds(allccds)
if cut is not None:
    allccds.cut(cut)
cut = survey.ccd_cuts(allccds)
allccds.cut(cut == 0)
print('Read {} CCDs.'.format(len(allccds)))

Converted brickname from |S8 to <U8
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-run19.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S51 to <U51
Converted plver from |S6 to <U6
Got 73860 CCDs
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-decals.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S61 to <U61
Converted plver from |S6 to <U6
Got 493440

In [ ]:
        # Read the parent super-sample.
        if False:
            print('Testing with smaller sample cuts!')
            d25min, d25max = 1.0, 5.0
            ramin, ramax = 175, 185
        else:
            d25min, d25max = 0.5, 10.0
            ramin, ramax = 0, 360

        cat = read_leda(largedir=largedir, d25min=d25min, d25max=d25max,
                        decmax=np.max(allccds.dec)+0.3,
                        decmin=np.min(allccds.dec)-0.3,


### Build the parent sample

In [15]:
def _build_sample_onegalaxy(args):
    """Filler function for the multiprocessing."""
    return build_sample_onegalaxy(*args)

In [16]:
def build_sample_onegalaxy(gal, allccds, ccdsdir, bricks, survey):
    """Wrapper function to find overlapping CCDs for a given galaxy.

    First generously find the nearest set of CCDs that are near the galaxy and
    then demand that there's 3-band coverage in a much smaller region centered
    on the galaxy.

    """
    #print('Working on {}...'.format(gal['GALAXY'].strip()))
    galwcs = _galwcs(gal)
    these = ccds_touching_wcs(galwcs, allccds)

    if len(these) > 0:
        ccds1 = _uniqccds( allccds[these] )

        # Is there 3-band coverage?
        galwcs_small = _galwcs(gal, factor=0.5)
        these_small = ccds_touching_wcs(galwcs_small, ccds1)
        ccds1_small = _uniqccds( ccds1[these_small] )

        if 'g' in ccds1_small.filter and 'r' in ccds1_small.filter and 'z' in ccds1_small.filter:
            print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Radius={:.4f} arcsec'.format(
                gal['GALAXY'].strip(), len(ccds1), gal['RA'], gal['DEC'], gal['RADIUS']))

            ccdsfile = os.path.join(ccdsdir, '{}-ccds.fits'.format(gal['GALAXY'].strip().lower()))
            #print('  Writing {}'.format(ccdsfile))
            if os.path.isfile(ccdsfile):
                os.remove(ccdsfile)
            ccds1.writeto(ccdsfile)

            # Also get the set of bricks touching this footprint.
            rad = 2*gal['RADIUS']/3600 # [degree]
            brickindx = survey.bricks_touching_radec_box(bricks,
                                                         gal['RA']-rad, gal['RA']+rad,
                                                         gal['DEC']-rad, gal['DEC']+rad)
            if len(brickindx) == 0 or len(brickindx) > 4:
                print('This should not happen!')
                pdb.set_trace()
            gal['BRICKNAME'][:len(brickindx)] = bricks.brickname[brickindx]

            return [gal, ccds1]

    return None


In [6]:
samplefile = os.path.join(LSLGAdir, 'sample', 'large-galaxies-{}.fits'.format(dr))
sample = fits.getdata(samplefile, 1)

'/global/cscratch1/sd/ioannis/repos/legacypipe'